# Eat Safe, Love

## Notebook Set Up

In [79]:
# Import dependencies
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [80]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [81]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [82]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [83]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [84]:
# Find the establishments with a hygiene score of 20
query = establishments.find({"scores.Hygiene": 20})

# Use count_documents to display the number of documents in the result
print("Number of establishments with a hygiene score of 20:", query.count())
# Display the first document in the results using pprint
pprint(query[0])

Number of establishments with a hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66011c7fd4d870011cff80e3'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'

/var/folders/bf/0vmhxl6x4xn54k_fw54k_rkr0000gn/T/ipykernel_17459/846555771.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print("Number of establishments with a hygiene score of 20:", query.count())


In [85]:
# Convert the result to a Pandas DataFrame
df_query = pd.DataFrame(list(query))
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df_query))
# Display the first 10 rows of the DataFrame
print(df_query.head(10))

Number of rows in the DataFrame: 41
                        _id   FHRSID  ChangesByServerID  \
0  66011c7fd4d870011cff80e3   110681                  0   
1  66011c7fd4d870011cff8466   612039                  0   
2  66011c7fd4d870011cff8773   730933                  0   
3  66011c7fd4d870011cff8960   172735                  0   
4  66011c7fd4d870011cff896d   172953                  0   
5  66011c7fd4d870011cff9310   512854                  0   
6  66011c7fd4d870011cff9531  1537089                  0   
7  66011c80d4d870011cffaa65   155648                  0   
8  66011c80d4d870011cffaea0  1012883                  0   
9  66011c80d4d870011cffb6b1   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532   

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [86]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = establishments.find({"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}})

# Use count_documents to display the number of documents in the result
count_query = query.count()
print("Number of establishments in London with RatingValue >= 4:", count_query)

# Display the first document in the results using pprint
if count_query > 0:
    pprint(next(query))
else:
    print("No establishments found.")


Number of establishments in London with RatingValue >= 4: 0
No establishments found.


/var/folders/bf/0vmhxl6x4xn54k_fw54k_rkr0000gn/T/ipykernel_17459/3717440569.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  count_query = query.count()


In [88]:
# Convert the result to a Pandas DataFrame
df_query = pd.DataFrame(list(query))
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df_query))
# Display the first 10 rows of the DataFrame
print(df_query.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [89]:
# Define the latitude and longitude range
latitude_range = {"$gte": 51.490142 - 0.01, "$lte": 51.490142 + 0.01}
longitude_range = {"$gte": 0.08384 - 0.01, "$lte": 0.08384 + 0.01}

# Query establishments within the specified latitude and longitude range, with RatingValue equals 5
nearest_establishments = establishments.find({
    "geocode.latitude": latitude_range,
    "geocode.longitude": longitude_range,
    "RatingValue": 5
}).sort("scores.Hygiene")

# Convert the result to a Pandas DataFrame
df_nearest_establishments = pd.DataFrame(list(nearest_establishments))

# Display the number of rows in the DataFrame
num_rows = df_nearest_establishments.shape[0]
print("Number of establishments:", num_rows)

# Display the DataFrame sorted by hygiene score
print("Establishments sorted by hygiene score:")
print(df_nearest_establishments)

Number of establishments: 87
Establishments sorted by hygiene score:
                         _id   FHRSID  ChangesByServerID  \
0   66011c80d4d870011cffd919   695223                  0   
1   66011c80d4d870011cffd92d  1380578                  0   
2   66011c80d4d870011cffd94c   694609                  0   
3   66011c80d4d870011cffd95e   695241                  0   
4   66011c80d4d870011cffd963   909912                  0   
..                       ...      ...                ...   
82  66011c80d4d870011cffdba3  1300359                  0   
83  66011c80d4d870011cffdbaa   694128                  0   
84  66011c80d4d870011cffdc89   947998                  0   
85  66011c80d4d870011cffdd42  1385142                  0   
86  66011c80d4d870011cffdd46  1156702                  0   

   LocalAuthorityBusinessID                                    BusinessName  \
0              PI/000178842                                         Iceland   
1                     14425             Howe and Co 

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude =
longitude =

query =
sort =
limit =

# Print the results


In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
